In [4]:
import numpy as np
import pandas as pd
import time 
import datetime as datetime
import glob
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
np.random.seed(1)
import gym
import gym_banana
import argparse
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch.autograd import Variable

In [5]:
Pad = pd.read_csv('PadData_v2.csv')
Pad.head()

,currency pair,timestamp,bid provider,bid price,bid volume,ask provider,ask price,ask volume,pad
0,AUDUSD,02.01.2019 00:00:00,LP-3,0.72714,1000000.0,LP-2,0.72718,2000000.0,0.0
1,AUDUSD,02.01.2019 00:00:01,LP-2,0.72712,2000000.0,LP-2,0.72717,2000000.0,0.0
2,AUDUSD,02.01.2019 00:00:02,LP-1,0.72713,1000000.0,LP-2,0.72718,2000000.0,0.0
3,AUDUSD,02.01.2019 00:00:03,LP-2,0.72713,1000000.0,LP-2,0.72717,2000000.0,0.0
4,AUDUSD,02.01.2019 00:00:04,LP-2,0.72713,2000000.0,LP-2,0.72717,1000000.0,0.0


In [5]:
T = 3617
m = 10
to_draw = np.sort(Pad['timestamp'].unique())
ccy = np.sort(Pad['currency pair'].unique())
min_history = 500 # min episode length

    
def generate_episode(n,cur):
    _max = to_draw.shape[0]
    _end = min(n+T, _max)
    timeframe = to_draw[n:_end]
    other_bid = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    other_ask = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    i = 0
    for elem in ccy:
        tmp = Pad[Pad['currency pair'] == elem]
        if elem == cur:
            target_bid = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            target_ask = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
        else:
            other_bid[:,i] = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            other_ask[:,i] = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
            i += 1
    return target_bid, target_ask, other_bid, other_ask

def features(price_path,m):
    features = np.zeros((price_path.shape[0]-m,m))
    for i in range(m):
        features[:,i] = (np.log(price_path) - np.log(np.roll(price_path, i+1)))[m:]
    return features

def get_features(target_bid, target_ask, other_bid, other_ask, m):
    feature_span = features(target_bid,m)
    feature_span = np.append(feature_span, features(target_ask,m), axis = 1)
    for i in range(other_bid.shape[1]):
        feature_span = np.append(feature_span, features(other_bid[:,i],m), axis = 1)
    for j in range(other_ask.shape[1]):
        feature_span = np.append(feature_span, features(other_ask[:,j],m), axis = 1)
    return feature_span

def draw_episode(m, cur, min_history):
    '''
    Input:
        m, number of lag returns z_1,...z_m
        cur, currency pair that we target to trade
        min_history, min length of a valid episode
    '''
#     n = np.random.randint(to_draw.shape[0] - min_history)
    n = 1
    target_bid, target_ask, other_bid, other_ask = generate_episode(n,cur)
    feature_span = get_features(target_bid, target_ask, other_bid, other_ask, m)
    return target_bid, target_ask, feature_span

def calculate_reward(previous_action, current_action):
    pass

def draw_episode_with_episode_num(m, cur, min_history, n = None):
    '''
    Input:
        m, number of lag returns z_1,...z_m
        cur, currency pair that we target to trade
        min_history, min length of a valid episode
    '''
    if n is None:
        n = np.random.randint(to_draw.shape[0] - min_history)
    
    target_bid, target_ask, other_bid, other_ask = generate_episode(n,cur)
    feature_span = get_features(target_bid, target_ask, other_bid, other_ask, m)
    return target_bid, target_ask, feature_span
    
def test_policy(policy = None, episode=None, totalNumEpisodes=1):
    if episode is None:
        for i in range(totalNumEpisodes):
            ask = np.zeros((1, 1))
            bid = np.zeros((1,1 ))
            previous_action = 0
            reward_for_this_episode = 0
            while ask.shape[0] <= 3600 and bid.shape[0]<=3600:
                target_bid, target_ask, feature_span = draw_episode_with_episode_num(16, 'AUDUSD', 1000, 1)
                bid, ask, features = target_bid[16:], target_ask[16:], feature_span
            for t in range(3600):  # Don't infinite loop while learning
                reward_for_this_episode += max(max(((ask[t+1] - ask[t]) + (bid[t+1] - bid[t]))/2, -((ask[t+1] - ask[t]) + (bid[t+1] - bid[t]))/2), 0)
            print("The reward for episode No.{} is {}".format(1, reward_for_this_episode))
            
        # a part of code to figure out the maximum value we can get
                
    else:
        #run policy through episode
        #compute
        pass
    
def main():
    test_policy()

if __name__ == '__main__':
    main()

The reward for episode No.1 is 0.04295500000004049


In [8]:


T = 3602
m = 10
to_draw = np.sort(Pad['timestamp'].unique())
ccy = np.sort(Pad['currency pair'].unique())
min_history = 500 # min episode length

    
def generate_episode(n,cur):
    _max = to_draw.shape[0]
    _end = min(n+T, _max)
    timeframe = to_draw[n:_end]
    other_bid = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    other_ask = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    i = 0
    for elem in ccy:
        tmp = Pad[Pad['currency pair'] == elem]
        if elem == cur:
            target_bid = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            target_ask = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
        else:
            other_bid[:,i] = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            other_ask[:,i] = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
            i += 1
    return target_bid, target_ask, other_bid, other_ask

def features(price_path,m):
    features = np.zeros((price_path.shape[0]-m,m))
    for i in range(m):
        features[:,i] = (np.log(price_path) - np.log(np.roll(price_path, i+1)))[m:]
    return features

def get_features(target_bid, target_ask, other_bid, other_ask, m):
    feature_span = features(target_bid,m)
    feature_span = np.append(feature_span, features(target_ask,m), axis = 1)
    for i in range(other_bid.shape[1]):
        feature_span = np.append(feature_span, features(other_bid[:,i],m), axis = 1)
    for j in range(other_ask.shape[1]):
        feature_span = np.append(feature_span, features(other_ask[:,j],m), axis = 1)
    return feature_span

def draw_episode(m, cur, min_history):
    '''
    Input:
        m, number of lag returns z_1,...z_m
        cur, currency pair that we target to trade
        min_history, min length of a valid episode
    '''
#     n = np.random.randint(to_draw.shape[0] - min_history)
    n = 8000
    target_bid, target_ask, other_bid, other_ask = generate_episode(n,cur)
    feature_span = get_features(target_bid, target_ask, other_bid, other_ask, m)
    normalized = (feature_span-feature_span.mean(axis=1))/feature_span.std(axis=1)
    return target_bid, target_ask, normalized
import gym
import gym_banana
import argparse
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch.autograd import Variable


#parser = argparse.ArgumentParser(description='PyTorch REINFORCE example')
# parser.add_argument('--gamma', type=float, default=0.99, metavar='G',
#                     help='discount factor (default: 0.99)')
# parser.add_argument('--seed', type=int, default=543, metavar='N',
#                     help='random seed (default: 543)')
# parser.add_argument('--render', action='store_true',
#                     help='render the environment')
# parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                     help='interval between training status logs (default: 10)')
#args = parser.parse_args()


env = gym.make('Banana-v0')
env.seed(1)
torch.manual_seed(1)


class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(16, 3, bias = True)
#         self.A = Variable(torch.randn(256, 3), requires_grad=True)
#         self.b = Variable(torch.randn(1), requires_grad=True)
        torch.nn.init.xavier_uniform(self.affine1.weight)
        #print('The weight of the affine layer is', self.affine1.weight)
#         self.dropout = nn.Dropout(p=0)
#        self.mu = Variable(torch.randn(3, 3), requires_grad=True)
        #self.mu = nn.Linear(1,3, bias = True)
                #torch.nn.init.xavier_uniform(self.affine1.weight)
        #print('The weight of the affine layer is', self.affine1.weight)
        # self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=-1)

        self.saved_log_probs = []
        self.rewards = 0
        self.actions = []

    def forward(self, x):
        x = self.affine1(x)
#         x = torch.matmul(x, self.A) 
#         x = self.dropout(x)
#         print(x.size())
        #mu_sigma = torch.matmul(y, self.mu)
#         print(mu_sigma.size())
        # action = self.tanh(x + mu_sigma)
        # action = self.softmax(action)
        action = self.softmax(x)
#         action_scores = self.affine2(x)
#         return F.softmax(action_scores, dim=1)
        return action


global policy
policy = Policy()

optimizer = optim.SGD(policy.parameters(), lr=1e-2)
eps = np.finfo(np.float32).eps.item()


# def select_action(state, previous_action):
#     state = torch.from_numpy(state).float()
#     previous_action = previous_action
#     probs = policy(state, previous_action)
    #return probs
    #m = Categorical(probs)
    #action = m.sample()
    #policy.saved_log_probs.append(m.log_prob(action))
#     return probs.detach().numpy()
    #return action.item() - 1


def finish_episode(num):
    R = 0
    policy_loss = []
    returns = []
#     for r in policy.rewards[::-1]:
#         #print("reward",r * 1000000)
# #         R = r
#         returns.insert(0, R)
#     returns = torch.tensor(returns)
#     optimizer.zero_grad()
#     # returns = (returns - returns.mean()) / (returns.std() + eps)
#     for R in returns:
# #         print(Variable(-torch.ones(log_prob.shape), requires_grad = True) * R)
#         #print("return", R)
#         #print("policy_append", Variable(-torch.ones(log_prob.shape) * 1000000, requires_grad = True) * R)
#         #policy_loss.append( torch.tensor(-1000000* R))
#         policy_loss.append(Variable(-torch.ones((1,)) * 1000000, requires_grad = True) * R)
 
#     policy_loss = torch.cat(policy_loss).sum() 
#     #policy_loss = policy.A.sum()
#     policy_loss.backward()
#     optimizer.step()
#     print(policy.A.grad)
#     #print(policy.b.grad)
#     #print(list(policy.parameters()))
#     print ("Finished {} episode and the policy loss is {}".format(num, policy_loss))
#     policy.rewards = []
    #optimizer.zero_grad()
    #del policy.saved|_log_probs[:]
    print(policy.A.grad)
    print(policy.b.grad)
    loss = policy.A.sum()
    loss.backward()
    policy.rewards.backward()
    optimizer.step()
    optimizer.zero_grad()
    


def main():
    mask = torch.tensor([-1, 0, 1]).float()
    for i_episode in range(100):
        ask = np.zeros((1, 1))
        bid = np.zeros((1,1 ))
        previous_action = np.array([0, 0, 1])
        while ask.shape[0] <= 3600 and bid.shape[0]<=3600:
            target_bid, target_ask, feature_span = draw_episode(1, 'AUDUSD', 1000)
            bid, ask, features = target_bid[1:]*1e5, target_ask[1:]*1e5, feature_span
        for t in range(3600):  # Don't infinite loop while learning
            state = feature_span[t]
#             action = select_action(state, previous_action)
            action = policy(torch.from_numpy(state).float())
            after_mask = action * mask
#             print('The action chosen at the current step is {}'.format(after_mask))
#             action = Variable(action, requires_grad=True)
#             reward = ((action) * (ask[t+1] - ask[t]) + (bid[t+1] - bid[t]))
#             reward = Variable(torch.tensor([reward.sum()]), requires_grad=True)
            price_change = (ask[t+1] - ask[t]) + (bid[t+1] - bid[t])
#             print ('The price change is {}'.format(price_change))
#             print('A preview of price change {}'.format(after_mask * price_change))
            reward = torch.sum(after_mask * price_change)
#             print('what we can get {}'.format(reward.float().item()))
#             print('The price chance at this time step is {}'.format(price_change))
            
            policy.rewards += reward
#             print('The cumulative reward so far is {}'.format(policy.rewards))
#             ep_reward += reward
            #policy.actions.append(action)
            previous_action = action


        
#         running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
#         finish_episode(i_episode)
#         print(policy.A.grad)
#         print(policy.b.grad)
        print(80*'=')
        print ('previous action {}'.format(previous_action))
        if policy.affine1.weight.grad is not None:
            print(policy.affine1.weight.grad)
            print('******weights******')
            print(policy.affine1.weight.data)
#         policy.rewards.backward()
#         loss = policy.A.sum()
        
        loss = - policy.rewards
        print (type(loss))
        print(loss.size())
        print(type(policy.rewards))
        print(torch.sum(policy.rewards).size())
        loss.backward(retain_graph=True)
        optimizer.step()
        print('The loss of training is {}'.format(loss.item()))
        policy.rewards = 0
#         optimizer.zero_grad()
#         if i_episode % 5 == 0:
#             print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
#                   i_episode, ep_reward, running_reward))
#         if running_reward > env.spec.reward_threshold:
#             print("Solved! Running reward is now {} and "
#                   "the last episode runs to {} time steps!".format(running_reward, t))
#             break


if __name__ == '__main__':
    main()

2019-04-27 23:47:47,324 - root - INFO - BananaEnv - Version 0.1.0


/Users/yilun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:92: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


previous action tensor([0.3459, 0.3617, 0.2924], grad_fn=<SoftmaxBackward>)
<class 'torch.Tensor'>
torch.Size([])
<class 'torch.Tensor'>
torch.Size([])
The loss of training is -151.17071533203125
previous action tensor([9.9994e-01, 5.8457e-05, 1.3623e-08], grad_fn=<SoftmaxBackward>)
tensor([[-3.9480e-02,  2.7206e-02,  6.3803e-04, -8.4873e-05, -5.0561e-02,
         -4.5311e-03, -1.0831e-03, -4.3460e-02, -7.4064e-03, -5.7956e-03,
          5.4978e-03,  3.3733e-02, -4.7377e-05, -1.5012e-03,  1.7507e-02,
          4.2726e-03],
        [ 2.3353e-03, -1.6093e-03, -3.7748e-05,  5.0805e-06,  2.9907e-03,
          2.6809e-04,  6.4044e-05,  2.5707e-03,  4.3813e-04,  3.4281e-04,
         -3.2527e-04, -1.9954e-03,  2.8389e-06,  8.8769e-05, -1.0356e-03,
         -2.5257e-04],
        [ 3.7145e-02, -2.5597e-02, -6.0028e-04,  7.9792e-05,  4.7570e-02,
          4.2630e-03,  1.0190e-03,  4.0890e-02,  6.9683e-03,  5.4528e-03,
         -5.1725e-03, -3.1738e-02,  4.4537e-05,  1.4124e-03, -1.6472e-02,
    

The loss of training is -2824.0
previous action tensor([1.0000e+00, 1.7487e-30, 0.0000e+00], grad_fn=<SoftmaxBackward>)
tensor([[-3.9487e-02,  2.7211e-02,  6.3814e-04, -8.4888e-05, -5.0570e-02,
         -4.5319e-03, -1.0833e-03, -4.3468e-02, -7.4077e-03, -5.7967e-03,
          5.4988e-03,  3.3739e-02, -4.7385e-05, -1.5014e-03,  1.7510e-02,
          4.2734e-03],
        [ 2.3423e-03, -1.6142e-03, -3.7862e-05,  5.0957e-06,  2.9998e-03,
          2.6890e-04,  6.4237e-05,  2.5785e-03,  4.3946e-04,  3.4384e-04,
         -3.2625e-04, -2.0015e-03,  2.8473e-06,  8.9037e-05, -1.0387e-03,
         -2.5334e-04],
        [ 3.7145e-02, -2.5597e-02, -6.0028e-04,  7.9792e-05,  4.7570e-02,
          4.2630e-03,  1.0190e-03,  4.0890e-02,  6.9683e-03,  5.4528e-03,
         -5.1725e-03, -3.1738e-02,  4.4537e-05,  1.4124e-03, -1.6472e-02,
         -4.0201e-03]])
******weights******
tensor([[-0.2652,  0.3030, -0.1366,  0.5597,  0.4540, -0.0260, -0.3750,  0.3452,
          0.1749, -0.3629,  0.3646,  0.3388

The loss of training is -2824.0
previous action tensor([1., 0., 0.], grad_fn=<SoftmaxBackward>)
tensor([[-3.9487e-02,  2.7211e-02,  6.3814e-04, -8.4888e-05, -5.0570e-02,
         -4.5319e-03, -1.0833e-03, -4.3468e-02, -7.4077e-03, -5.7967e-03,
          5.4988e-03,  3.3739e-02, -4.7385e-05, -1.5014e-03,  1.7510e-02,
          4.2734e-03],
        [ 2.3423e-03, -1.6142e-03, -3.7862e-05,  5.0957e-06,  2.9998e-03,
          2.6890e-04,  6.4237e-05,  2.5785e-03,  4.3946e-04,  3.4384e-04,
         -3.2625e-04, -2.0015e-03,  2.8473e-06,  8.9037e-05, -1.0387e-03,
         -2.5334e-04],
        [ 3.7145e-02, -2.5597e-02, -6.0028e-04,  7.9792e-05,  4.7570e-02,
          4.2630e-03,  1.0190e-03,  4.0890e-02,  6.9683e-03,  5.4528e-03,
         -5.1725e-03, -3.1738e-02,  4.4537e-05,  1.4124e-03, -1.6472e-02,
         -4.0201e-03]])
******weights******
tensor([[-0.2628,  0.3014, -0.1367,  0.5597,  0.4570, -0.0257, -0.3750,  0.3479,
          0.1754, -0.3625,  0.3643,  0.3368,  0.4984, -0.3148, -0.0

...

The loss of training is -2824.0
previous action tensor([1., 0., 0.], grad_fn=<SoftmaxBackward>)
tensor([[-3.9487e-02,  2.7211e-02,  6.3814e-04, -8.4888e-05, -5.0570e-02,
         -4.5319e-03, -1.0833e-03, -4.3468e-02, -7.4077e-03, -5.7967e-03,
          5.4988e-03,  3.3739e-02, -4.7385e-05, -1.5014e-03,  1.7510e-02,
          4.2734e-03],
        [ 2.3423e-03, -1.6142e-03, -3.7862e-05,  5.0957e-06,  2.9998e-03,
          2.6890e-04,  6.4237e-05,  2.5785e-03,  4.3946e-04,  3.4384e-04,
         -3.2625e-04, -2.0015e-03,  2.8473e-06,  8.9037e-05, -1.0387e-03,
         -2.5334e-04],
        [ 3.7145e-02, -2.5597e-02, -6.0028e-04,  7.9792e-05,  4.7570e-02,
          4.2630e-03,  1.0190e-03,  4.0890e-02,  6.9683e-03,  5.4528e-03,
         -5.1725e-03, -3.1738e-02,  4.4537e-05,  1.4124e-03, -1.6472e-02,
         -4.0201e-03]])
******weights******
tensor([[-0.2391,  0.2851, -0.1371,  0.5598,  0.4874, -0.0230, -0.3743,  0.3739,
          0.1798, -0.3590,  0.3610,  0.3165,  0.4984, -0.3139, -0.1

The loss of training is -2824.0
previous action tensor([1., 0., 0.], grad_fn=<SoftmaxBackward>)
tensor([[-3.9487e-02,  2.7211e-02,  6.3814e-04, -8.4888e-05, -5.0570e-02,
         -4.5319e-03, -1.0833e-03, -4.3468e-02, -7.4077e-03, -5.7967e-03,
          5.4988e-03,  3.3739e-02, -4.7385e-05, -1.5014e-03,  1.7510e-02,
          4.2734e-03],
        [ 2.3423e-03, -1.6142e-03, -3.7862e-05,  5.0957e-06,  2.9998e-03,
          2.6890e-04,  6.4237e-05,  2.5785e-03,  4.3946e-04,  3.4384e-04,
         -3.2625e-04, -2.0015e-03,  2.8473e-06,  8.9037e-05, -1.0387e-03,
         -2.5334e-04],
        [ 3.7145e-02, -2.5597e-02, -6.0028e-04,  7.9792e-05,  4.7570e-02,
          4.2630e-03,  1.0190e-03,  4.0890e-02,  6.9683e-03,  5.4528e-03,
         -5.1725e-03, -3.1738e-02,  4.4537e-05,  1.4124e-03, -1.6472e-02,
         -4.0201e-03]])
******weights******
tensor([[-0.2367,  0.2834, -0.1371,  0.5598,  0.4904, -0.0227, -0.3742,  0.3765,
          0.1803, -0.3587,  0.3607,  0.3145,  0.4984, -0.3138, -0.1

The loss of training is -2824.0
previous action tensor([1., 0., 0.], grad_fn=<SoftmaxBackward>)
tensor([[-3.9487e-02,  2.7211e-02,  6.3814e-04, -8.4888e-05, -5.0570e-02,
         -4.5319e-03, -1.0833e-03, -4.3468e-02, -7.4077e-03, -5.7967e-03,
          5.4988e-03,  3.3739e-02, -4.7385e-05, -1.5014e-03,  1.7510e-02,
          4.2734e-03],
        [ 2.3423e-03, -1.6142e-03, -3.7862e-05,  5.0957e-06,  2.9998e-03,
          2.6890e-04,  6.4237e-05,  2.5785e-03,  4.3946e-04,  3.4384e-04,
         -3.2625e-04, -2.0015e-03,  2.8473e-06,  8.9037e-05, -1.0387e-03,
         -2.5334e-04],
        [ 3.7145e-02, -2.5597e-02, -6.0028e-04,  7.9792e-05,  4.7570e-02,
          4.2630e-03,  1.0190e-03,  4.0890e-02,  6.9683e-03,  5.4528e-03,
         -5.1725e-03, -3.1738e-02,  4.4537e-05,  1.4124e-03, -1.6472e-02,
         -4.0201e-03]])
******weights******
tensor([[-0.2344,  0.2818, -0.1371,  0.5598,  0.4934, -0.0225, -0.3742,  0.3792,
          0.1807, -0.3583,  0.3603,  0.3125,  0.4984, -0.3137, -0.1

The loss of training is -2824.0
previous action tensor([1., 0., 0.], grad_fn=<SoftmaxBackward>)
tensor([[-3.9487e-02,  2.7211e-02,  6.3814e-04, -8.4888e-05, -5.0570e-02,
         -4.5319e-03, -1.0833e-03, -4.3468e-02, -7.4077e-03, -5.7967e-03,
          5.4988e-03,  3.3739e-02, -4.7385e-05, -1.5014e-03,  1.7510e-02,
          4.2734e-03],
        [ 2.3423e-03, -1.6142e-03, -3.7862e-05,  5.0957e-06,  2.9998e-03,
          2.6890e-04,  6.4237e-05,  2.5785e-03,  4.3946e-04,  3.4384e-04,
         -3.2625e-04, -2.0015e-03,  2.8473e-06,  8.9037e-05, -1.0387e-03,
         -2.5334e-04],
        [ 3.7145e-02, -2.5597e-02, -6.0028e-04,  7.9792e-05,  4.7570e-02,
          4.2630e-03,  1.0190e-03,  4.0890e-02,  6.9683e-03,  5.4528e-03,
         -5.1725e-03, -3.1738e-02,  4.4537e-05,  1.4124e-03, -1.6472e-02,
         -4.0201e-03]])
******weights******
tensor([[-0.2320,  0.2802, -0.1372,  0.5598,  0.4965, -0.0222, -0.3741,  0.3818,
          0.1811, -0.3580,  0.3600,  0.3105,  0.4984, -0.3136, -0.1

The loss of training is -2824.0
previous action tensor([1., 0., 0.], grad_fn=<SoftmaxBackward>)
tensor([[-3.9487e-02,  2.7211e-02,  6.3814e-04, -8.4888e-05, -5.0570e-02,
         -4.5319e-03, -1.0833e-03, -4.3468e-02, -7.4077e-03, -5.7967e-03,
          5.4988e-03,  3.3739e-02, -4.7385e-05, -1.5014e-03,  1.7510e-02,
          4.2734e-03],
        [ 2.3423e-03, -1.6142e-03, -3.7862e-05,  5.0957e-06,  2.9998e-03,
          2.6890e-04,  6.4237e-05,  2.5785e-03,  4.3946e-04,  3.4384e-04,
         -3.2625e-04, -2.0015e-03,  2.8473e-06,  8.9037e-05, -1.0387e-03,
         -2.5334e-04],
        [ 3.7145e-02, -2.5597e-02, -6.0028e-04,  7.9792e-05,  4.7570e-02,
          4.2630e-03,  1.0190e-03,  4.0890e-02,  6.9683e-03,  5.4528e-03,
         -5.1725e-03, -3.1738e-02,  4.4537e-05,  1.4124e-03, -1.6472e-02,
         -4.0201e-03]])
******weights******
tensor([[-0.2296,  0.2785, -0.1372,  0.5598,  0.4995, -0.0219, -0.3740,  0.3844,
          0.1816, -0.3576,  0.3597,  0.3084,  0.4984, -0.3136, -0.1

In [ ]:


T = 3602
m = 10
to_draw = np.sort(Pad['timestamp'].unique())
ccy = np.sort(Pad['currency pair'].unique())
min_history = 500 # min episode length

    
def generate_episode(n,cur):
    _max = to_draw.shape[0]
    _end = min(n+T, _max)
    timeframe = to_draw[n:_end]
    other_bid = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    other_ask = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    i = 0
    for elem in ccy:
        tmp = Pad[Pad['currency pair'] == elem]
        if elem == cur:
            target_bid = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            target_ask = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
        else:
            other_bid[:,i] = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            other_ask[:,i] = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
            i += 1
    return target_bid, target_ask, other_bid, other_ask

def features(price_path,m):
    features = np.zeros((price_path.shape[0]-m,m))
    for i in range(m):
        features[:,i] = (np.log(price_path) - np.log(np.roll(price_path, i+1)))[m:]
    return features

def get_features(target_bid, target_ask, other_bid, other_ask, m):
    feature_span = features(target_bid,m)
    feature_span = np.append(feature_span, features(target_ask,m), axis = 1)
    for i in range(other_bid.shape[1]):
        feature_span = np.append(feature_span, features(other_bid[:,i],m), axis = 1)
    for j in range(other_ask.shape[1]):
        feature_span = np.append(feature_span, features(other_ask[:,j],m), axis = 1)
    return feature_span

def draw_episode(m, cur, min_history):
    '''
    Input:
        m, number of lag returns z_1,...z_m
        cur, currency pair that we target to trade
        min_history, min length of a valid episode
    '''
    n = np.random.randint(to_draw.shape[0] - min_history)
    n = 5000
    target_bid, target_ask, other_bid, other_ask = generate_episode(n,cur)
    feature_span = get_features(target_bid, target_ask, other_bid, other_ask, m)
    normalized = (feature_span-feature_span.mean())/feature_span.std()
    return target_bid, target_ask, normalized
import gym
import gym_banana
import argparse
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch.autograd import Variable


#parser = argparse.ArgumentParser(description='PyTorch REINFORCE example')
# parser.add_argument('--gamma', type=float, default=0.99, metavar='G',
#                     help='discount factor (default: 0.99)')
# parser.add_argument('--seed', type=int, default=543, metavar='N',
#                     help='random seed (default: 543)')
# parser.add_argument('--render', action='store_true',
#                     help='render the environment')
# parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                     help='interval between training status logs (default: 10)')
#args = parser.parse_args()


env = gym.make('Banana-v0')
env.seed(1)
torch.manual_seed(1)


class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(16, 1, bias = True)
#         self.A = Variable(torch.randn(256, 3), requires_grad=True)
#         self.b = Variable(torch.randn(1), requires_grad=True)
        torch.nn.init.xavier_uniform(self.affine1.weight)
        #print('The weight of the affine layer is', self.affine1.weight)
#         self.dropout = nn.Dropout(p=0)
#        self.mu = Variable(torch.randn(3, 3), requires_grad=True)
        #self.mu = nn.Linear(1,3, bias = True)
                #torch.nn.init.xavier_uniform(self.affine1.weight)
        #print('The weight of the affine layer is', self.affine1.weight)
        # self.tanh = nn.Tanh()
        self.tanh = nn.Tanh()

        self.saved_log_probs = []
        self.rewards = 0
        self.actions = []

    def forward(self, x):
        x = self.affine1(x)
#         x = torch.matmul(x, self.A) 
#         x = self.dropout(x)
#         print(x.size())
        #mu_sigma = torch.matmul(y, self.mu)
#         print(mu_sigma.size())
        # action = self.tanh(x + mu_sigma)
        # action = self.softmax(action)
        action = self.tanh(x)
#         action_scores = self.affine2(x)
#         return F.softmax(action_scores, dim=1)
        return action


global policy
policy = Policy()

optimizer = optim.SGD(policy.parameters(), lr=1e-1)
eps = np.finfo(np.float32).eps.item()


# def select_action(state, previous_action):
#     state = torch.from_numpy(state).float()
#     previous_action = previous_action
#     probs = policy(state, previous_action)
    #return probs
    #m = Categorical(probs)
    #action = m.sample()
    #policy.saved_log_probs.append(m.log_prob(action))
#     return probs.detach().numpy()
    #return action.item() - 1


def finish_episode(num):
    R = 0
    policy_loss = []
    returns = []
#     for r in policy.rewards[::-1]:
#         #print("reward",r * 1000000)
# #         R = r
#         returns.insert(0, R)
#     returns = torch.tensor(returns)
#     optimizer.zero_grad()
#     # returns = (returns - returns.mean()) / (returns.std() + eps)
#     for R in returns:
# #         print(Variable(-torch.ones(log_prob.shape), requires_grad = True) * R)
#         #print("return", R)
#         #print("policy_append", Variable(-torch.ones(log_prob.shape) * 1000000, requires_grad = True) * R)
#         #policy_loss.append( torch.tensor(-1000000* R))
#         policy_loss.append(Variable(-torch.ones((1,)) * 1000000, requires_grad = True) * R)
 
#     policy_loss = torch.cat(policy_loss).sum() 
#     #policy_loss = policy.A.sum()
#     policy_loss.backward()
#     optimizer.step()
#     print(policy.A.grad)
#     #print(policy.b.grad)
#     #print(list(policy.parameters()))
#     print ("Finished {} episode and the policy loss is {}".format(num, policy_loss))
#     policy.rewards = []
    #optimizer.zero_grad()
    #del policy.saved|_log_probs[:]
    print(policy.A.grad)
    print(policy.b.grad)
    loss = policy.A.sum()
    loss.backward()
    policy.rewards.backward()
    optimizer.step()
    optimizer.zero_grad()
    


def main():
    for i_episode in range(100):
        ask = np.zeros((1, 1))
        bid = np.zeros((1,1 ))
        previous_action = np.array([0, 0, 1])
        while ask.shape[0] <= 3600 and bid.shape[0]<=3600:
            target_bid, target_ask, feature_span = draw_episode(1, 'AUDUSD', 1000)
            bid, ask, features = target_bid[1:]*1e3, target_ask[1:]*1e3, feature_span
        for t in range(3600):  # Don't infinite loop while learning
            state = feature_span[t]
#             action = select_action(state, previous_action)
            action = policy(torch.from_numpy(state).float())
#             print('The action chosen at the current step is {}'.format(action))
#             action = Variable(action, requires_grad=True)
#             reward = ((action) * (ask[t+1] - ask[t]) + (bid[t+1] - bid[t]))
#             reward = Variable(torch.tensor([reward.sum()]), requires_grad=True)
            price_change = (ask[t] - ask[t+1]) + (bid[t] - bid[t+1])
#             print ('The price change is {}'.format(price_change))
#             print('A preview of price change {}'.format(after_mask * price_change))
            reward = torch.sum(action * price_change)
#             print('what we can get {}'.format(reward.float().item()))
#             print('The price chance at this time step is {}'.format(price_change))
            
            policy.rewards += reward
#             print('The cumulative reward so far is {}'.format(policy.rewards))
#             ep_reward += reward
            #policy.actions.append(action)
            previous_action = action


        
#         running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
#         finish_episode(i_episode)
#         print(policy.A.grad)
#         print(policy.b.grad)
        print(80*'=')
        print ('previous action {}'.format(previous_action))
        if policy.affine1.weight.grad is not None:
            print(policy.affine1.weight.grad)
            print('******weights******')
            print(policy.affine1.weight.data)
#         policy.rewards.backward()
#         loss = policy.A.sum()
        optimizer.zero_grad()
        loss = - policy.rewards / 3600
        print (type(loss))
        print(loss.size())
        print(type(policy.rewards))
        print(torch.sum(policy.rewards).size())
        loss.backward(retain_graph=True)
        optimizer.step()
        print('The loss of training is {}'.format(loss.item()))
        policy.rewards = 0
#         optimizer.zero_grad()
#         if i_episode % 5 == 0:
#             print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
#                   i_episode, ep_reward, running_reward))
#         if running_reward > env.spec.reward_threshold:
#             print("Solved! Running reward is now {} and "
#                   "the last episode runs to {} time steps!".format(running_reward, t))
#             break


if __name__ == '__main__':
    main()

2019-04-28 00:41:34,673 - root - INFO - BananaEnv - Version 0.1.0


/Users/yilun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:93: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


previous action tensor([-0.0230], grad_fn=<TanhBackward>)
<class 'torch.Tensor'>
torch.Size([])
<class 'torch.Tensor'>
torch.Size([])
The loss of training is 6.741825200151652e-05
previous action tensor([-0.0221], grad_fn=<TanhBackward>)
tensor([[-3.2275e-04, -4.8325e-04,  8.6255e-05,  1.0477e-03,  8.8287e-05,
          4.4157e-04, -1.7045e-04, -1.3647e-03, -3.4695e-04, -2.2551e-07,
         -4.0916e-04,  2.7266e-04,  2.4595e-04,  7.9067e-06,  1.2086e-03,
          5.4806e-04]])
******weights******
tensor([[-0.0023,  0.5194,  0.1849, -0.2213, -0.3588, -0.0996, -0.2562, -0.1902,
          0.0285,  0.3542,  0.3229, -0.5808,  0.3683,  0.1660,  0.5634,  0.3921]])
<class 'torch.Tensor'>
torch.Size([])
<class 'torch.Tensor'>
torch.Size([])
The loss of training is 5.881975084776059e-05
previous action tensor([-0.0211], grad_fn=<TanhBackward>)
tensor([[-3.2309e-04, -4.8353e-04,  8.6207e-05,  1.0478e-03,  8.8069e-05,
          4.4163e-04, -1.7036e-04, -1.3648e-03, -3.4683e-04, -2.8953e-07,
    

previous action tensor([-0.0112], grad_fn=<TanhBackward>)
tensor([[-3.2684e-04, -4.8659e-04,  8.5667e-05,  1.0488e-03,  8.5652e-05,
          4.4225e-04, -1.6943e-04, -1.3666e-03, -3.4540e-04, -9.9918e-07,
         -4.0983e-04,  2.6974e-04,  2.4714e-04,  7.9313e-06,  1.2101e-03,
          5.4772e-04]])
******weights******
tensor([[-0.0020,  0.5200,  0.1848, -0.2226, -0.3589, -0.1002, -0.2560, -0.1886,
          0.0289,  0.3542,  0.3234, -0.5811,  0.3680,  0.1660,  0.5620,  0.3914]])
<class 'torch.Tensor'>
torch.Size([])
<class 'torch.Tensor'>
torch.Size([])
The loss of training is -4.440671909833327e-05
previous action tensor([-0.0103], grad_fn=<TanhBackward>)
tensor([[-3.2718e-04, -4.8687e-04,  8.5616e-05,  1.0489e-03,  8.5430e-05,
          4.4231e-04, -1.6934e-04, -1.3667e-03, -3.4527e-04, -1.0642e-06,
         -4.0988e-04,  2.6949e-04,  2.4724e-04,  7.9335e-06,  1.2102e-03,
          5.4768e-04]])
******weights******
tensor([[-0.0019,  0.5200,  0.1848, -0.2227, -0.3589, -0.1002, -0

previous action tensor([-0.0003], grad_fn=<TanhBackward>)
tensor([[-3.3092e-04, -4.8991e-04,  8.5053e-05,  1.0496e-03,  8.2976e-05,
          4.4286e-04, -1.6838e-04, -1.3681e-03, -3.4379e-04, -1.7839e-06,
         -4.1044e-04,  2.6676e-04,  2.4832e-04,  7.9600e-06,  1.2111e-03,
          5.4727e-04]])
******weights******
tensor([[-0.0016,  0.5205,  0.1847, -0.2239, -0.3590, -0.1007, -0.2558, -0.1870,
          0.0293,  0.3542,  0.3239, -0.5814,  0.3677,  0.1659,  0.5605,  0.3908]])
<class 'torch.Tensor'>
torch.Size([])
<class 'torch.Tensor'>
torch.Size([])
The loss of training is -0.00014768035907763988
previous action tensor([0.0006], grad_fn=<TanhBackward>)
tensor([[-3.3126e-04, -4.9018e-04,  8.5001e-05,  1.0496e-03,  8.2752e-05,
          4.4291e-04, -1.6829e-04, -1.3682e-03, -3.4365e-04, -1.8498e-06,
         -4.1049e-04,  2.6651e-04,  2.4841e-04,  7.9626e-06,  1.2112e-03,
          5.4723e-04]])
******weights******
tensor([[-0.0015,  0.5206,  0.1847, -0.2240, -0.3590, -0.1007, -0

The loss of training is -0.0002423423429718241
previous action tensor([0.0105], grad_fn=<TanhBackward>)
tensor([[-3.3500e-04, -4.9320e-04,  8.4414e-05,  1.0500e-03,  8.0262e-05,
          4.4339e-04, -1.6729e-04, -1.3693e-03, -3.4213e-04, -2.5793e-06,
         -4.1101e-04,  2.6371e-04,  2.4947e-04,  7.9927e-06,  1.2119e-03,
          5.4672e-04]])
******weights******
tensor([[-0.0012,  0.5211,  0.1846, -0.2251, -0.3591, -0.1012, -0.2556, -0.1853,
          0.0297,  0.3542,  0.3244, -0.5817,  0.3674,  0.1659,  0.5590,  0.3901]])
<class 'torch.Tensor'>
torch.Size([])
<class 'torch.Tensor'>
torch.Size([])
The loss of training is -0.0002509458572603762
previous action tensor([0.0114], grad_fn=<TanhBackward>)
tensor([[-3.3534e-04, -4.9347e-04,  8.4360e-05,  1.0501e-03,  8.0034e-05,
          4.4344e-04, -1.6720e-04, -1.3694e-03, -3.4198e-04, -2.6460e-06,
         -4.1105e-04,  2.6346e-04,  2.4957e-04,  7.9955e-06,  1.2119e-03,
          5.4667e-04]])
******weights******
tensor([[-0.0011,  0

The loss of training is -0.0003455550177022815
previous action tensor([0.0213], grad_fn=<TanhBackward>)
tensor([[-3.3908e-04, -4.9647e-04,  8.3751e-05,  1.0502e-03,  7.7510e-05,
          4.4385e-04, -1.6617e-04, -1.3701e-03, -3.4041e-04, -3.3848e-06,
         -4.1153e-04,  2.6061e-04,  2.5061e-04,  8.0288e-06,  1.2123e-03,
          5.4607e-04]])
******weights******
tensor([[-0.0008,  0.5217,  0.1845, -0.2264, -0.3592, -0.1017, -0.2554, -0.1837,
          0.0301,  0.3542,  0.3249, -0.5821,  0.3671,  0.1659,  0.5576,  0.3895]])
<class 'torch.Tensor'>
torch.Size([])
<class 'torch.Tensor'>
torch.Size([])
The loss of training is -0.00035415092133916914
previous action tensor([0.0223], grad_fn=<TanhBackward>)
tensor([[-3.3942e-04, -4.9674e-04,  8.3695e-05,  1.0502e-03,  7.7279e-05,
          4.4388e-04, -1.6608e-04, -1.3702e-03, -3.4026e-04, -3.4521e-06,
         -4.1157e-04,  2.6034e-04,  2.5071e-04,  8.0320e-06,  1.2123e-03,
          5.4601e-04]])
******weights******
tensor([[-0.0007,  